In [1]:
import pandas as pd
def print_ratio(df,x_values,save_name):
    # 计算每个组合的平均return
    avg_return = df.groupby(['algorithm', 'train mode', 'test mode'])['ep ret'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_return['algorithm'].unique()

    for algo in algorithms:
        algo_data = avg_return[avg_return['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        for x in x_values:
            before_condition = (avg_return['train mode'] == x - 1) & (avg_return['test mode'] == x - 1)
            max_ret = avg_return[before_condition]['ep ret'].max()
            
            # condition after training on new task
            after_condition = (algo_data['train mode'] == x) & (algo_data['test mode'] == x - 1)
            after_resutls = algo_data[after_condition]
            
            # condition before training on new task
            before_condition = (algo_data['train mode'] == x - 1) & (algo_data['test mode'] == x - 1)
            before_results = algo_data[before_condition]
            
            if not after_resutls.empty and not before_results.empty:
                performance_before = before_results['ep ret'].values[0] / max_ret
                performance_after = after_resutls['ep ret'].values[0] / max_ret
                # ratio = (performance_before - performance_after)
                ratio = (performance_before - performance_after) / performance_before
                if algo in ['MaskNet', "PackNet"]:
                    print(performance_before, performance_after)
                    print(ratio,ratio / performance_before)
                # ratio = (performance_before - performance_after) 
                algo_ratios[f'forgetting on {x}'] = ratio 
            else:
                algo_ratios[f'forgetting on {x}'] = None  
        
        # algo_ratios_df = pd.DataFrame(algo_ratios)
        # result_df = pd.concat([result_df, algo_ratios_df], ignore_index=True)
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = numeric_df.mean(axis=1)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)
    
def print_final_performance(df,x_values,save_name):
    # 计算每个组合的平均return
    avg_return = df.groupby(['algorithm', 'train mode', 'test mode'])['ep ret'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_return['algorithm'].unique()
    train_mode = x_values[-1]
    for algo in algorithms:
        algo_data = avg_return[avg_return['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        # 遍历每个x值
        for x in x_values:
            cur_condition = (avg_return['train mode'] == x) & (avg_return['test mode'] == x)
            max_ret = avg_return[cur_condition]['ep ret'].max()
            
            # 筛选出train mode = x, test mode = x-1的行
            condition1 = (algo_data['train mode'] == train_mode) & (algo_data['test mode'] == x)
            result1 = algo_data[condition1]
            
            # 筛选出train mode = x-1, test mode = x-1的行
            prev_condition = (algo_data['train mode'] == x) & (algo_data['test mode'] == x)
            result2 = algo_data[prev_condition]
            
            # 如果两组数据都存在，则计算比值
            if not result1.empty and not result2.empty:
                ratio = result1['ep ret'].values[0] / max_ret
                algo_ratios[f'{x}'] = ratio  # 动态列名
            else:
                algo_ratios[f'{x}'] = 0  # 如果没有匹配的数据，则设为None
        
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = numeric_df.mean(axis=1)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)

In [2]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('eval_Freeway/eval_results.csv')
x_values = range(1,8)
print_ratio(df, x_values, 'eval_Freeway/forgetting_results')

# 读取CSV文件
df = pd.read_csv('eval_SpaceInvaders/eval_results.csv')
x_values = range(1,10)
print_ratio(df, x_values, 'eval_SpaceInvaders/forgetting_results')

/tmp/ipykernel_76135/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:40: FutureWarning: The fr

0.9291666666666667 0.9458333333333333
-0.01793721973094164 -0.019304631100564993
0.15677966101694915 0.1271186440677966
0.18918918918918923 1.2067202337472611
0.3508771929824561 0.12719298245614033
0.6375000000000001 1.8168750000000002
0.424 0.22399999999999998
0.47169811320754723 1.1124955500178002
0.6759259259259259 0.7037037037037037
-0.04109589041095892 -0.060799399512103613
0.3436123348017621 0.2290748898678414
0.3333333333333333 0.97008547008547
0.26200873362445415 0.2183406113537118
0.16666666666666663 0.636111111111111
0.9333333333333332 0.9291666666666667
0.00446428571428558 0.004783163265305979
0.1271186440677966 0.1271186440677966
0.0 0.0
0.07456140350877193 0.06578947368421052
0.11764705882352941 1.5778546712802768
0.36 0.08
0.7777777777777777 2.1604938271604937
0.6759259259259259 0.6450617283950617
0.045662100456621044 0.06755488834678182
0.3392070484581498 0.3480176211453745
-0.02597402597402601 -0.07657277787147927
0.24017467248908297 0.27947598253275113
-0.1636363636363

/tmp/ipykernel_76135/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:40: FutureWarning: The fr

In [6]:
import pandas as pd

# 读取CSV文件
# df = pd.read_csv('eval_Freeway/eval_results.csv')
# for x in range(1,9):
#     x_values = range(x)
#     print_final_performance(df, x_values, f'eval_Freeway/{x}_final_perf_results')

df = pd.read_csv('eval_SpaceInvaders/eval_results.csv')
for x in range(1,11):
    x_values = range(x)
    print_final_performance(df, x_values, f'eval_SpaceInvaders/{x}_final_perf_results')

/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The fr

  algorithm         0      mean
0    CReLUs  0.726415  0.726415
1    CbpNet  0.729846  0.729846
2  CompoNet  0.849914  0.849914
3        F1  0.987993  0.987993
4        FN  1.000000  1.000000
5   FuseNet  0.803602  0.803602
6   MaskNet  0.232419  0.232419
7   PackNet  0.529160  0.529160
8   ProgNet  0.823328  0.823328
  algorithm         0         1      mean
0    CReLUs  0.788165  0.897092  0.842628
1    CbpNet  0.825901  0.804623  0.815262
2  CompoNet  0.666381  0.747204  0.706792
3        F1  0.845626  0.890380  0.868003
4        FN  0.955403  0.993289  0.974346
5   FuseNet  0.819039  1.000000  0.909520
6   MaskNet  0.310463  0.278151  0.294307
7   PackNet  0.409949  0.331842  0.370895
8   ProgNet  0.305317  0.746458  0.525888
  algorithm         0         1         2      mean
0    CReLUs  0.808748  0.452647  0.775453  0.678949
1    CbpNet  0.696398  0.458613  0.762901  0.639304
2  CompoNet  0.850772  0.437733  1.000000  0.762835
3        F1  0.520583  0.459359  0.630404  0.536782


/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The fr

  algorithm         0         1         2         3      mean
0    CReLUs  0.561750  0.588367  0.904463  0.827798  0.720594
1    CbpNet  0.576329  0.741238  0.742678  0.948954  0.752300
2  CompoNet  0.498285  0.548844  0.887029  1.000000  0.733540
3        F1  0.674957  0.753169  0.617155  0.677737  0.680754
4        FN  0.433962  0.649515  1.129707  0.808733  0.755479
5   FuseNet  0.588336  0.680835  0.831241  0.730627  0.707760
6   MaskNet  0.262436  0.211037  0.267782  0.220787  0.240510
7   PackNet  0.467410  0.346010  0.228033  0.215252  0.314177
8   ProgNet  0.626930  0.536167  0.600418  0.572571  0.584021
  algorithm         0         1         2         3         4      mean
0    CReLUs  0.733276  0.382550  0.914226  0.604551  0.864569  0.699835
1    CbpNet  0.691252  0.362416  0.943515  0.458180  0.725034  0.636079
2  CompoNet  0.549743  0.269202  0.960948  0.587946  1.000000  0.673568
3        F1  0.849057  0.329605  0.408647  0.276138  0.854309  0.543551
4        FN  0.47427

/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The fr

  algorithm         0         1         2         3         4         5  \
0    CReLUs  0.530017  0.366890  0.671548  0.688807  1.106019  0.635201   
1    CbpNet  0.636364  0.277405  0.791492  0.527675  0.849521  0.283191   
2  CompoNet  0.777873  0.427293  1.063459  0.709102  1.097811  0.701583   
3        F1  0.788165  0.549590  0.781032  0.455105  0.426129  0.135201   
4        FN  0.775300  0.326622  1.261506  0.833948  1.364569  0.587089   
5   FuseNet  0.566895  0.374348  0.994421  0.751538  0.896717  0.626066   
6   MaskNet  0.538593  0.227442  0.251743  0.211562  0.129275  0.064555   
7   PackNet  0.450257  0.317673  0.332636  0.183887  0.185363  0.049939   
8   ProgNet  0.613208  0.189411  0.705718  0.277983  0.500684  0.056029   

          6      mean  
0  0.610772  0.658465  
1  0.547256  0.558986  
2  0.695122  0.781749  
3  0.697663  0.547555  
4  1.000000  0.878434  
5  0.776931  0.712416  
6  0.128049  0.221603  
7  0.071138  0.227270  
8  0.378049  0.388726  
  algorit

/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_76135/2137582870.py:81: FutureWarning: The fr